In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_rows', 500)

In [83]:
perek_body = pd.read_excel('../parsed_data/perek_categories.xlsx').query('green_light == 1').reset_index(drop=True)

In [84]:
lvl1_lst = perek_body.groupby('level_1', as_index=False).count().query('level_0>1')['level_1'].unique().tolist()

In [85]:
perek_body[perek_body['level_1'].isin(lvl1_lst)].sort_values(by='level_1')

,level_0,level_1,level_1_href,green_light
105,Мясо и птица,Деликатесы и копчёности,https://www.perekrestok.ru/cat/c/136/delikates...,1
117,Колбасные изделия,Деликатесы и копчёности,https://www.perekrestok.ru/cat/c/809/delikates...,1
110,Мясо и птица,Заливное,https://www.perekrestok.ru/cat/c/781/zalivnoe,1
120,Колбасные изделия,Заливное,https://www.perekrestok.ru/cat/c/810/zalivnoe,1
170,"Сухие завтраки, мюсли",Каши,https://www.perekrestok.ru/cat/c/172/kasi,1
175,Продукты быстрого приготовления,Каши,https://www.perekrestok.ru/cat/c/756/kasi,1
109,Мясо и птица,Паштеты,https://www.perekrestok.ru/cat/c/778/pastety,1
119,Колбасные изделия,Паштеты,https://www.perekrestok.ru/cat/c/808/pastety,1
111,Мясо и птица,Студни,https://www.perekrestok.ru/cat/c/779/studni,1
121,Колбасные изделия,Студни,https://www.perekrestok.ru/cat/c/811/studni,1


In [86]:
perek_head = pd.read_csv('perek_item_sample.csv', sep='\t')

In [87]:
full_perek = perek_body.merge(perek_head, on='level_1_href', how='inner').drop(columns=['green_light'])
full_perek

,level_0,level_1,level_1_href,level_2,item_name,value
0,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Премиум меню,Cырники из рикотты с фруктовым соусом Шеф Пере...,149.99
1,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Премиум меню,"Пшенная каша с манго, персиком и эстрагоном Ше...",139.99
2,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Каша,"Каша молочная рисовая Шеф Перекрёсток, 250г",169.99
3,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Каша,Каша рисовая на кокосовом молоке с фруктами Ше...,169.99
4,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Каша,"Каша пшённая на молоке Шеф Перекресток, 250г",169.99
...,...,...,...,...,...,...
12872,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.45,Коктейль Черный Русский с коньяком и вкусом ми...,109.99
12873,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.45,Напиток слабоалкогольный Очаково Джин грейпфру...,99.99
12874,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.45,Напиток слабоалкогольный Очаково Джин-тоник 9%...,99.99
12875,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.5,"Коктейль алкогольный Pre яблоко 20%, 500мл",539.99


In [95]:
same_cat_name = full_perek[full_perek['level_1'] == full_perek['level_2']]['level_1'].unique()
same_cat_name

array(['Десерты', 'Хрен', 'Какао', 'Горячий шоколад', 'Бренди'],
      dtype=object)

In [27]:
perek_lst = full_perek[~full_perek['level_2'].isin(['Рекомендуем'])].groupby('item_name', as_index=False)\
        .count().query('level_0 > 1')['item_name'].tolist()
full_perek[full_perek['item_name'].isin(perek_lst)].sort_values('item_name')[:10]

,level_0,level_1,level_1_href,level_2,item_name,value
8543,Консервация,Рыбные консервы,https://www.perekrestok.ru/cat/c/75/rybnye-kon...,Остальное,Анчоус Delicius европейский филе в подсолнечно...,279.99
6519,Рыба,Рыбные консервы и кулинария,https://www.perekrestok.ru/cat/c/181/rybnye-ko...,Остальное,Анчоус Delicius европейский филе в подсолнечно...,279.99
6211,Рыба,Вся рыба,https://www.perekrestok.ru/cat/c/812/vsa-ryba,Остальное,Анчоус Delicius европейский филе в подсолнечно...,279.99
6135,Рыба,Вся рыба,https://www.perekrestok.ru/cat/c/812/vsa-ryba,Остальное,"Анчоус Сухогруз сушёный, 70г",149.99
6535,Рыба,Сушёная рыба,https://www.perekrestok.ru/cat/c/178/susenaa-ryba,Остальное,"Анчоус Сухогруз сушёный, 70г",149.99
8208,Консервация,Овощные консервы,https://www.perekrestok.ru/cat/c/76/ovosnye-ko...,Томаты,"Ассорти Дядя Ваня огурцы и томаты, 1.8кг",439.99
8174,Консервация,Овощные консервы,https://www.perekrestok.ru/cat/c/76/ovosnye-ko...,Огурцы,"Ассорти Дядя Ваня огурцы и томаты, 1.8кг",439.99
8213,Консервация,Овощные консервы,https://www.perekrestok.ru/cat/c/76/ovosnye-ko...,Томаты,Ассорти Скатерть-Самобранка из корнишонов и то...,219.99
8179,Консервация,Овощные консервы,https://www.perekrestok.ru/cat/c/76/ovosnye-ko...,Огурцы,Ассорти Скатерть-Самобранка из корнишонов и то...,219.99
3497,"Шоколад, конфеты, сладости",Батончики,https://www.perekrestok.ru/cat/c/203/batonciki,Фруктовый,Батончик Take A Bite Макадамия-Ваниль орехово-...,139.99


In [29]:
help_df = full_perek[['level_0', 'level_1', 'level_2']].drop_duplicates()\
        .groupby(['level_0', 'level_1'], as_index=False).count().query('level_2 == 1')
lst_lvl1 = help_df['level_1'].unique().tolist()
lst_lvl0 = help_df['level_0'].unique().tolist()
help_df.shape

(26, 3)

In [30]:
levels_df = full_perek[['level_0', 'level_1', 'level_2']].drop_duplicates()
indexes = full_perek[(full_perek['level_0'].isin(lst_lvl0)) & (full_perek['level_1'].isin(lst_lvl1))].index
full_perek.loc[indexes, 'level_2'] = full_perek.loc[indexes, 'level_1']

In [53]:
full_perek = full_perek[~full_perek['level_2'].isin(['Рекомендуем'])].reset_index(drop=True)
full_perek.groupby('item_name', as_index=False)\
        .count().query('level_0 > 1')['item_name'].tolist()
# Список дублированных item_nameов 
item_lst = full_perek[full_perek['item_name'].isin(perek_lst)]\
            .sort_values('item_name')\
            .query('level_2 != "Остальное"')['item_name'].unique().tolist()
double_index = full_perek[full_perek['item_name'].isin(item_lst)].query('level_2 == "Остальное"').index
full_perek['mark'] = 1
full_perek.loc[double_index, 'mark'] = 0
full_perek

,level_0,level_1,level_1_href,level_2,item_name,value,mark
0,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Премиум меню,Cырники из рикотты с фруктовым соусом Шеф Пере...,149.99,1
1,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Премиум меню,"Пшенная каша с манго, персиком и эстрагоном Ше...",139.99,1
2,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Каша,"Каша молочная рисовая Шеф Перекрёсток, 250г",169.99,1
3,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Каша,Каша рисовая на кокосовом молоке с фруктами Ше...,169.99,1
4,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Каша,"Каша пшённая на молоке Шеф Перекресток, 250г",169.99,1
...,...,...,...,...,...,...,...
10051,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.45,Коктейль Черный Русский с коньяком и вкусом ми...,109.99,1
10052,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.45,Напиток слабоалкогольный Очаково Джин грейпфру...,99.99,1
10053,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.45,Напиток слабоалкогольный Очаково Джин-тоник 9%...,99.99,1
10054,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.5,"Коктейль алкогольный Pre яблоко 20%, 500мл",539.99,1


In [71]:
full_perek_filtered = full_perek[full_perek['mark'] == 1]

perek_lst = full_perek_filtered.groupby('item_name', as_index=False)\
        .count().query('level_0 > 1')['item_name'].tolist()
doubles_without_other = full_perek_filtered[full_perek_filtered['item_name'].isin(perek_lst)]\
        .sort_values('item_name').index
        
full_perek_filtered.loc[doubles_without_other, 'mark'] = 2
full_perek_filtered[full_perek_filtered['item_name'].isin(perek_lst)]\
        .sort_values('item_name')

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,level_0,level_1,level_1_href,level_2,item_name,value,mark


In [88]:
after_modification = pd.read_excel('../modified_data/full_perek.xlsx', sheet_name='Лист2')
after_modification

,level_0,level_1,level_1_href,level_2,strange_elements,selected_depth(от 0),selected_level
0,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Премиум меню,1.0,2.0,Премиум меню
1,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Каша,NaN,2.0,Каша
2,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Блинчики,NaN,2.0,Блинчики
3,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Сырники,NaN,2.0,Сырники
4,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,Запеканка,NaN,2.0,Запеканка
...,...,...,...,...,...,...,...
886,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.33,1.0,1.0,Слабоалкогольные напитки
887,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.36,1.0,1.0,Слабоалкогольные напитки
888,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.45,1.0,1.0,Слабоалкогольные напитки
889,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,0.5,1.0,1.0,Слабоалкогольные напитки


In [118]:
a = after_modification.merge(perek_head, on=['level_1_href', 'level_2'], how='inner').drop(columns=['selected_depth(от 0)',
                                                                                               'level_2'])
a

,level_0,level_1,level_1_href,strange_elements,selected_level,item_name,value
0,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,1.0,Премиум меню,Cырники из рикотты с фруктовым соусом Шеф Пере...,149.99
1,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,1.0,Премиум меню,"Пшенная каша с манго, персиком и эстрагоном Ше...",139.99
2,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,NaN,Каша,"Каша молочная рисовая Шеф Перекрёсток, 250г",169.99
3,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,NaN,Каша,Каша рисовая на кокосовом молоке с фруктами Ше...,169.99
4,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,NaN,Каша,"Каша пшённая на молоке Шеф Перекресток, 250г",169.99
...,...,...,...,...,...,...,...
9781,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,Коктейль Черный Русский с коньяком и вкусом ми...,109.99
9782,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,Напиток слабоалкогольный Очаково Джин грейпфру...,99.99
9783,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,Напиток слабоалкогольный Очаково Джин-тоник 9%...,99.99
9784,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,"Коктейль алкогольный Pre яблоко 20%, 500мл",539.99


In [119]:
a = a.drop_duplicates()
a

,level_0,level_1,level_1_href,strange_elements,selected_level,item_name,value
0,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,1.0,Премиум меню,Cырники из рикотты с фруктовым соусом Шеф Пере...,149.99
1,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,1.0,Премиум меню,"Пшенная каша с манго, персиком и эстрагоном Ше...",139.99
2,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,NaN,Каша,"Каша молочная рисовая Шеф Перекрёсток, 250г",169.99
3,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,NaN,Каша,Каша рисовая на кокосовом молоке с фруктами Ше...,169.99
4,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,NaN,Каша,"Каша пшённая на молоке Шеф Перекресток, 250г",169.99
...,...,...,...,...,...,...,...
9781,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,Коктейль Черный Русский с коньяком и вкусом ми...,109.99
9782,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,Напиток слабоалкогольный Очаково Джин грейпфру...,99.99
9783,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,Напиток слабоалкогольный Очаково Джин-тоник 9%...,99.99
9784,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,"Коктейль алкогольный Pre яблоко 20%, 500мл",539.99


In [120]:
lst = a.groupby('item_name', as_index=False)\
        .count().query('level_0 > 1')['item_name'].tolist()

indexes = a[a['item_name'].isin(lst)].sort_values('item_name').index
a['mark'] = 1
a.loc[a['selected_level'] == 'Остальное', 'mark'] = 0
a.loc[indexes, 'mark'] = 2
a

C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,level_0,level_1,level_1_href,strange_elements,selected_level,item_name,value,mark
0,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,1.0,Премиум меню,Cырники из рикотты с фруктовым соусом Шеф Пере...,149.99,1
1,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,1.0,Премиум меню,"Пшенная каша с манго, персиком и эстрагоном Ше...",139.99,1
2,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,NaN,Каша,"Каша молочная рисовая Шеф Перекрёсток, 250г",169.99,1
3,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,NaN,Каша,Каша рисовая на кокосовом молоке с фруктами Ше...,169.99,1
4,Готовая еда,Завтрак,https://www.perekrestok.ru/cat/c/27/zavtrak,NaN,Каша,"Каша пшённая на молоке Шеф Перекресток, 250г",169.99,1
...,...,...,...,...,...,...,...,...
9781,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,Коктейль Черный Русский с коньяком и вкусом ми...,109.99,1
9782,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,Напиток слабоалкогольный Очаково Джин грейпфру...,99.99,1
9783,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,Напиток слабоалкогольный Очаково Джин-тоник 9%...,99.99,1
9784,Алкогольные напитки,Слабоалкогольные напитки,https://www.perekrestok.ru/cat/c/11/slaboalkog...,1.0,Слабоалкогольные напитки,"Коктейль алкогольный Pre яблоко 20%, 500мл",539.99,1


In [121]:
a.to_excel('../modified_data/full_perek_marks2.xlsx', index=False)

C:\Program Files\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.9.6' currently installed).
  warnings.warn(msg, UserWarning)


In [ ]:
categorizer[categorizer['']]

In [97]:
categorizer = pd.read_excel('../modified_data/full_perek_marks2.xlsx', sheet_name='Лист6')
categorizer.head()

,target_level_1,target_level_2,target_level_3,target_level_4,near_product,validation_item_name,validation_level,ref,x
0,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_00] Бальзам,[05_00_00_00] Бальзам,[05_00_00_00_00] Бальзам,"Бальзам МИКСТУРА Биттер 35%, 0.5л, Россия, 0.5 L",4,https://www.perekrestok.ru/cat/c/17/balzam,Бальзам
1,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_01] Бренди,[05_00_01_00] Бренди,[05_00_01_00_00] Бренди,Дербентская крепость Нап.крепкий алкогольный б...,4,https://www.perekrestok.ru/cat/c/14/brendi,NaN
2,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_02] Вермут,[05_00_02_00] Вермут,[05_00_02_00_00] Вермут,"САНТО СТЕФАНО ВЕРМУТ 500 мл,",4,https://www.perekrestok.ru/cat/c/764/vermut,NaN
3,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_03] Вино,[05_00_03_00] Вино,[05_00_03_00_00] Вино,"ВиноТорчикода Саленто2020крсух 0,75л Италия",4,https://www.perekrestok.ru/cat/c/1048/vina-rossii,Вина России
4,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_03] Вино,[05_00_03_00] Вино,[05_00_03_00_00] Вино,472010Грядка 0.450.82м Красное вино 3005,4,https://www.perekrestok.ru/cat/c/2/vino,NaN


In [98]:
perek_items = pd.read_csv('perek_item_sample.csv', sep='\t')\
    .rename(columns={'level_1_href': 'ref', 'level_2': 'x'}).drop_duplicates()

perek_items['refx'] = perek_items.apply(lambda x: x['ref'] + ('|'+x['x'] if x['x'] else ''), axis=1)
perek_items.head()

,ref,x,item_name,value,refx
0,https://www.perekrestok.ru/cat/c/1038/pivo,Крафт,Пивной напиток Konix Belgian Dubbel пастеризов...,124.99,https://www.perekrestok.ru/cat/c/1038/pivo|Крафт
1,https://www.perekrestok.ru/cat/c/1038/pivo,Крафт,Пивной напиток Konix Belgian Blonde Ale пастер...,124.99,https://www.perekrestok.ru/cat/c/1038/pivo|Крафт
2,https://www.perekrestok.ru/cat/c/1038/pivo,Крафт,Пиво Troll Brew IPA светлое нефильтрованное 7....,99.99,https://www.perekrestok.ru/cat/c/1038/pivo|Крафт
3,https://www.perekrestok.ru/cat/c/1038/pivo,Крафт,"Пиво Афанасий Крафтовое светлое 4.5%, 750мл",199.99,https://www.perekrestok.ru/cat/c/1038/pivo|Крафт
4,https://www.perekrestok.ru/cat/c/1038/pivo,Крафт,"Пиво Bowler Ipa 6.1%, 500мл",179.99,https://www.perekrestok.ru/cat/c/1038/pivo|Крафт


In [105]:
merge_first = categorizer.merge(perek_items[~perek_items['x'].isin(same_cat_name)], on=['ref', 'x'], how='left')
merge_first.loc[merge_first['item_name'].isna(), 'x'] = np.nan

second_part = merge_first[merge_first['item_name'].isna()].drop(columns=['x', 'item_name', 'value', 'refx'])

merge_first.fillna(value={'x': ''}, inplace=True)
merge_first['refx'] = merge_first.apply(lambda x: x['ref'] + ('|'+x['x'] if x['x'] else ''), axis=1)
exc_list = merge_first[merge_first['x'] != '']['refx'].unique().tolist()
merge_first.drop(columns=['ref', 'x'], inplace=True)

merge_first

,target_level_1,target_level_2,target_level_3,target_level_4,near_product,validation_item_name,validation_level,item_name,value,refx
0,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_00] Бальзам,[05_00_00_00] Бальзам,[05_00_00_00_00] Бальзам,"Бальзам МИКСТУРА Биттер 35%, 0.5л, Россия, 0.5 L",4,NaN,NaN,https://www.perekrestok.ru/cat/c/17/balzam
1,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_01] Бренди,[05_00_01_00] Бренди,[05_00_01_00_00] Бренди,Дербентская крепость Нап.крепкий алкогольный б...,4,NaN,NaN,https://www.perekrestok.ru/cat/c/14/brendi
2,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_02] Вермут,[05_00_02_00] Вермут,[05_00_02_00_00] Вермут,"САНТО СТЕФАНО ВЕРМУТ 500 мл,",4,NaN,NaN,https://www.perekrestok.ru/cat/c/764/vermut
3,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_03] Вино,[05_00_03_00] Вино,[05_00_03_00_00] Вино,"ВиноТорчикода Саленто2020крсух 0,75л Италия",4,NaN,NaN,https://www.perekrestok.ru/cat/c/1048/vina-rossii
4,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_03] Вино,[05_00_03_00] Вино,[05_00_03_00_00] Вино,472010Грядка 0.450.82м Красное вино 3005,4,NaN,NaN,https://www.perekrestok.ru/cat/c/2/vino
...,...,...,...,...,...,...,...,...,...,...
959,[05] Продукты (FOOD),"[05_12] Шоколад, конфеты, сладости",[05_12_13] Пирожные,[05_12_13_00] Пирожные,[05_12_13_00_00] Пирожные,Профитроли творожные пирожное 350г,4,NaN,NaN,https://www.perekrestok.ru/cat/c/704/piroznye
960,[05] Продукты (FOOD),"[05_12] Шоколад, конфеты, сладости",[05_12_14] Пряники,[05_12_14_00] Пряники,[05_12_14_00_00] Пряники,Мак. Пряники Зебра лебедевская 350г12шт.,4,NaN,NaN,https://www.perekrestok.ru/cat/c/703/praniki
961,[05] Продукты (FOOD),"[05_12] Шоколад, конфеты, сладости",[05_12_15] Торты,[05_12_15_00] Торты,[05_12_15_00_00] Торты,ТОРТТ ПРАЗДНИЧНЫЙ 1300ГР ДАРИ ВКУС,4,NaN,NaN,https://www.perekrestok.ru/cat/c/201/torty
962,[05] Продукты (FOOD),"[05_12] Шоколад, конфеты, сладости",[05_12_16] Шоколад,[05_12_16_00] Шоколад,[05_12_16_00_00] Шоколад,шок. бел Alpen Gold малина 85 гр,4,NaN,NaN,https://www.perekrestok.ru/cat/c/195/sokolad


In [106]:
merge_second = second_part.merge(perek_items[~perek_items['refx'].isin(exc_list)], on='ref')
merge_second['refx'] = merge_second['ref']
merge_second.drop(columns=['ref'], inplace=True)
merge_second

,target_level_1,target_level_2,target_level_3,target_level_4,near_product,validation_item_name,validation_level,x,item_name,value,refx
0,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_00] Бальзам,[05_00_00_00] Бальзам,[05_00_00_00_00] Бальзам,"Бальзам МИКСТУРА Биттер 35%, 0.5л, Россия, 0.5 L",4,Отечественный,"Бальзам Ruskeala Дикая вишня 35%, 500мл",499.99,https://www.perekrestok.ru/cat/c/17/balzam
1,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_00] Бальзам,[05_00_00_00] Бальзам,[05_00_00_00_00] Бальзам,"Бальзам МИКСТУРА Биттер 35%, 0.5л, Россия, 0.5 L",4,Отечественный,"Бальзам Ruskeala Крафт курант 35%, 500мл",619.99,https://www.perekrestok.ru/cat/c/17/balzam
2,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_00] Бальзам,[05_00_00_00] Бальзам,[05_00_00_00_00] Бальзам,"Бальзам МИКСТУРА Биттер 35%, 0.5л, Россия, 0.5 L",4,Отечественный,"Бальзам Бугульма 40%, 500мл",419.99,https://www.perekrestok.ru/cat/c/17/balzam
3,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_00] Бальзам,[05_00_00_00] Бальзам,[05_00_00_00_00] Бальзам,"Бальзам МИКСТУРА Биттер 35%, 0.5л, Россия, 0.5 L",4,Импортный,"Бальзам Riga Black Balsam Чёрный 45%, 500мл",1 129.00,https://www.perekrestok.ru/cat/c/17/balzam
4,[05] Продукты (FOOD),[05_00] Алкогольные напитки,[05_00_00] Бальзам,[05_00_00_00] Бальзам,[05_00_00_00_00] Бальзам,"Бальзам МИКСТУРА Биттер 35%, 0.5л, Россия, 0.5 L",4,Остальное,Настойка на коньяке Сормовская Клюква сладкая ...,249.99,https://www.perekrestok.ru/cat/c/17/balzam
...,...,...,...,...,...,...,...,...,...,...,...
10315,[05] Продукты (FOOD),"[05_12] Шоколад, конфеты, сладости",[05_12_17] Шоколадные и ореховые пасты,[05_12_17_00] Шоколадные и ореховые пасты,[05_12_17_00_00] Шоколадные и ореховые пасты,Nutella паста ореховаяс добавлением какао 350г...,4,Урбеч,Паста Урбеч Биопродукты из семян льна натураль...,229.99,https://www.perekrestok.ru/cat/c/204/sokoladny...
10316,[05] Продукты (FOOD),"[05_12] Шоколад, конфеты, сладости",[05_12_17] Шоколадные и ореховые пасты,[05_12_17_00] Шоколадные и ореховые пасты,[05_12_17_00_00] Шоколадные и ореховые пасты,Nutella паста ореховаяс добавлением какао 350г...,4,Остальное,Паста карамельная Jean Rene Оригинальная солён...,269.99,https://www.perekrestok.ru/cat/c/204/sokoladny...
10317,[05] Продукты (FOOD),"[05_12] Шоколад, конфеты, сладости",[05_12_17] Шоколадные и ореховые пасты,[05_12_17_00] Шоколадные и ореховые пасты,[05_12_17_00_00] Шоколадные и ореховые пасты,Nutella паста ореховаяс добавлением какао 350г...,4,Остальное,"Паста Nattys семечки подсолнуха с мёдом, 325г",229.99,https://www.perekrestok.ru/cat/c/204/sokoladny...
10318,[05] Продукты (FOOD),"[05_12] Шоколад, конфеты, сладости",[05_12_17] Шоколадные и ореховые пасты,[05_12_17_00] Шоколадные и ореховые пасты,[05_12_17_00_00] Шоколадные и ореховые пасты,Nutella паста ореховаяс добавлением какао 350г...,4,Остальное,"Паста зефирная Крем-Зефир Классический, 130г",69.99,https://www.perekrestok.ru/cat/c/204/sokoladny...


In [107]:
result = merge_first.append(merge_second)\
        .sort_values(['target_level_1', 'target_level_2', 
                      'target_level_3', 'target_level_4', 'near_product'])\
    .drop_duplicates()
result = result[~result['item_name'].isna()].reset_index(drop=True)\
    .rename(columns={'refx': 'ref'})
result.to_excel('perek_result.xlsx', index=False)

C:\Program Files\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.9.6' currently installed).
  warnings.warn(msg, UserWarning)


In [108]:
result[['target_level_1', 'target_level_2', 'target_level_3', 'target_level_4', 
        'near_product', 'validation_item_name', 'validation_level', 'ref']]\
    .drop_duplicates().to_excel('tc_FOOD.xlsx', index=False)

C:\Program Files\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '0.9.8' or newer of 'xlsxwriter' (version '0.9.6' currently installed).
  warnings.warn(msg, UserWarning)
